In [15]:
install.packages('glmnet')
install.packages('caret')
library(glmnet)
library(caret)

Installing package into ‘/home/lian6/R/x86_64-pc-linux-gnu-library/3.1’
(as ‘lib’ is unspecified)
Installing package into ‘/home/lian6/R/x86_64-pc-linux-gnu-library/3.1’
(as ‘lib’ is unspecified)
also installing the dependencies ‘numDeriv’, ‘KernSmooth’, ‘lava’, ‘kernlab’, ‘CVST’, ‘DEoptimR’, ‘survival’, ‘prodlim’, ‘DRR’, ‘robustbase’, ‘sfsmisc’, ‘glue’, ‘ipred’, ‘dimRed’, ‘ddalpha’, ‘rlang’, ‘gower’, ‘RcppRoll’, ‘tidyselect’, ‘recipes’, ‘withr’

Loading required package: lattice
Loading required package: ggplot2


In [1]:
train = read.csv("train.csv", stringsAsFactors = FALSE)
test = read.csv("test.csv", stringsAsFactors = FALSE)

In [2]:
train = train[, -1]
test = test [,-1]

In [3]:
train$SalePrice <- log(train$SalePrice + 1)

In [4]:
#Create NA level for variables that have many NA
#train
for (x in c("Alley", "PoolQC", "MiscFeature", "Fence", "FireplaceQu", "GarageType", 
            "GarageFinish", "GarageQual", "GarageCond", "BsmtQual", "BsmtCond", 
            "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "MasVnrType")) {
  train[is.na(train[, x]), x] = "None"
}

#test
for (x in c("Alley", "PoolQC", "MiscFeature", "Fence", "FireplaceQu", "GarageType", 
            "GarageFinish", "GarageQual", "GarageCond", "BsmtQual", "BsmtCond", 
            "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "MasVnrType")) {
  test[is.na(test[, x]), x] = "None"
}

In [5]:
# Lotfront data from the mean of local neibour:

#train
mnm = aggregate(LotFrontage ~ Neighborhood, data = train, mean)
mnm2 = c()
for (str in train$Neighborhood[is.na(train$LotFrontage)]) {
  mnm2 = c(mnm2, which(mnm$Neighborhood == str))
}
train$LotFrontage[is.na(train$LotFrontage)] = mnm[mnm2, 2]

#test
mnm = aggregate(LotFrontage ~ Neighborhood, data = test, mean)
mnm2 = c()
for (str in test$Neighborhood[is.na(test$LotFrontage)]) {
  mnm2 = c(mnm2, which(mnm$Neighborhood == str))
}
test$LotFrontage[is.na(test$LotFrontage)] = mnm[mnm2, 2]

In [6]:
## Replacing numerical data with 0

#train
for (col in c( "BsmtFinSF2", "BsmtUnfSF", "BsmtFullBath", "BsmtHalfBath","GarageArea", "GarageCars", "BsmtFinSF1", 
              "TotalBsmtSF", "MasVnrArea")) {
  train[is.na(train[, col]), col] = 0
}

#test
for (col in c( "BsmtFinSF2", "BsmtUnfSF", "BsmtFullBath", "BsmtHalfBath","GarageArea", "GarageCars", "BsmtFinSF1", 
              "TotalBsmtSF", "MasVnrArea")) {
  test[is.na(test[, col]), col] = 0
}


In [7]:
#replacing NA for variables with having NA with the mode:
test$MSZoning[is.na(test$MSZoning)] = "RL"

test$Functional[is.na(test$Functional)] = "Typ"

test$Electrical[is.na(test$Electrical)] = "SBrkr"
train$Electrical[is.na(train$Electrical)] = "SBrkr"

test$KitchenQual[is.na(test$KitchenQual)] = "TA"

test$SaleType[is.na(test$SaleType)] = "WD"

test$Exterior1st[is.na(test$Exterior1st)] = "VinylSd"
test$Exterior2nd[is.na(test$Exterior2nd)] = "VinylSd"

In [8]:
## Remove Utilities as it has zero variance
test = test[, -9]
train = train[, -9]

In [9]:
#replace gargeyrbuilt with yrbuilt
i1 <- which(is.na(train$GarageYrBlt))
train[i1, 'GarageYrBlt'] <- train[i1, 'YearBuilt']

i2 <- which(is.na(test$GarageYrBlt))
test[i2, 'GarageYrBlt'] <- test[i2, 'YearBuilt']

In [12]:
#check whether there is NA in data
colSums(is.na(train))
colSums(is.na(test))

MSSubClass      MSZoning   LotFrontage       LotArea        Street 
            0             0             0             0             0 
        Alley      LotShape   LandContour     LotConfig     LandSlope 
            0             0             0             0             0 
 Neighborhood    Condition1    Condition2      BldgType    HouseStyle 
            0             0             0             0             0 
  OverallQual   OverallCond     YearBuilt  YearRemodAdd     RoofStyle 
            0             0             0             0             0 
     RoofMatl   Exterior1st   Exterior2nd    MasVnrType    MasVnrArea 
            0             0             0             0             0 
    ExterQual     ExterCond    Foundation      BsmtQual      BsmtCond 
            0             0             0             0             0 
 BsmtExposure  BsmtFinType1    BsmtFinSF1  BsmtFinType2    BsmtFinSF2 
            0             0             0             0             0 
    BsmtUnfSF   TotalBsmtSF       Heating     HeatingQC    CentralAir 
            0             0             0             0             0 
   Electrical     X1stFlrSF     X2ndFlrSF  LowQualFinSF     GrLivArea 
            0             0             0             0             0 
 BsmtFullBath  BsmtHalfBath      FullBath      HalfBath  BedroomAbvGr 
            0             0             0             0             0 
 KitchenAbvGr   KitchenQual  TotRmsAbvGrd    Functional    Fireplaces 
            0             0             0             0             0 
  FireplaceQu    GarageType   GarageYrBlt  GarageFinish    GarageCars 
            0             0             0             0             0 
   GarageArea    GarageQual    GarageCond    PavedDrive    WoodDeckSF 
            0             0             0             0             0 
  OpenPorchSF EnclosedPorch    X3SsnPorch   ScreenPorch      PoolArea 
            0             0             0             0             0 
       PoolQC         Fence   MiscFeature       MiscVal        MoSold 
            0             0             0             0             0 
       YrSold      SaleType SaleCondition     SalePrice 
            0             0             0             0

MSSubClass      MSZoning   LotFrontage       LotArea        Street 
            0             0             0             0             0 
        Alley      LotShape   LandContour     LotConfig     LandSlope 
            0             0             0             0             0 
 Neighborhood    Condition1    Condition2      BldgType    HouseStyle 
            0             0             0             0             0 
  OverallQual   OverallCond     YearBuilt  YearRemodAdd     RoofStyle 
            0             0             0             0             0 
     RoofMatl   Exterior1st   Exterior2nd    MasVnrType    MasVnrArea 
            0             0             0             0             0 
    ExterQual     ExterCond    Foundation      BsmtQual      BsmtCond 
            0             0             0             0             0 
 BsmtExposure  BsmtFinType1    BsmtFinSF1  BsmtFinType2    BsmtFinSF2 
            0             0             0             0             0 
    BsmtUnfSF   TotalBsmtSF       Heating     HeatingQC    CentralAir 
            0             0             0             0             0 
   Electrical     X1stFlrSF     X2ndFlrSF  LowQualFinSF     GrLivArea 
            0             0             0             0             0 
 BsmtFullBath  BsmtHalfBath      FullBath      HalfBath  BedroomAbvGr 
            0             0             0             0             0 
 KitchenAbvGr   KitchenQual  TotRmsAbvGrd    Functional    Fireplaces 
            0             0             0             0             0 
  FireplaceQu    GarageType   GarageYrBlt  GarageFinish    GarageCars 
            0             0             0             0             0 
   GarageArea    GarageQual    GarageCond    PavedDrive    WoodDeckSF 
            0             0             0             0             0 
  OpenPorchSF EnclosedPorch    X3SsnPorch   ScreenPorch      PoolArea 
            0             0             0             0             0 
       PoolQC         Fence   MiscFeature       MiscVal        MoSold 
            0             0             0             0             0 
       YrSold      SaleType SaleCondition 
            0             0             0

In [13]:
# replace some numerical data into catagorical data
train$MSSubClass = as.character(train$MSSubClass)
train$OverallCond = as.character(train$OverallCond)
train$MoSold = as.character(train$MoSold)
train$YrSold = as.character(train$YrSold)
test$MSSubClass = as.character(test$MSSubClass)
test$OverallCond = as.character(test$OverallCond)
test$MoSold = as.character(test$MoSold)
test$YrSold = as.character(test$YrSold)


In [16]:
#create dummies variable

#train
feature <- sapply(names(train), function(x) {
  class(train[[x]])
})
numeric <- names(feature[feature != "character"])
categorical <- names(feature[feature == "character"])

dummies <- dummyVars(~., train[categorical])
categorical_dummy <- predict(dummies, train[categorical])

train <- cbind(train[numeric], categorical_dummy)

In [19]:
#test
feature <- sapply(names(test), function(x) {
  class(test[[x]])
})
numeric <- names(feature[feature != "character"])
categorical <- names(feature[feature == "character"])

dummies <- dummyVars(~., test[categorical])
categorical_dummy <- predict(dummies, test[categorical])

test <- cbind(test[numeric], categorical_dummy)

In [20]:
# let train and test dataset have same variables
for (i in names(train)){
  if (!(i %in% names(test))) {
    test[paste(i)] <-rep(0,nrow(test))
  }
}

for (i in names(test)){
  if (!(i %in% names(train))) {
    train[paste(i)] <-rep(0,nrow(train))
  }
}

In [21]:
#lasso
set.seed(123)
cv_lasso = cv.glmnet(as.matrix(train[, -33]), train[, 33])
pre.las <- predict(cv_lasso, newx = as.matrix(test[,-322]), s = "lambda.min")

In [24]:
summary(exp(pre.las)-1)


       1         
 Min.   : 12635  
 1st Qu.: 95081  
 Median :118560  
 Mean   :134763  
 3rd Qu.:162352  
 Max.   :852433  